In [1]:
#definimos librerias para utilizar.

#debemos instalar Selenium "pip install selenium"
# Docs https://selenium-python.readthedocs.io
#debemos instalar el manejador del driver "pip install webdriver-manager"

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd 
import numpy as np

In [2]:
#inicializamos el navegador usado por Selenium, para esto necesitamos un Web Driver.

chrome_options = webdriver.ChromeOptions();
chrome_options.add_argument("--incognito");
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options);

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 29.1MB/s]


In [4]:
#hacemos el request a la pagina.
#en este caso pre filtramos la localidad de Usaquen y tomamos la url de la pagina que consultamos
driver.get("https://www.metrocuadrado.com/apartamento-casa/venta/bogota/usaquen/?search=form");
time.sleep(3);

In [5]:
#identificamos la paginación, el boton de siguiente y traemos el total de registros
pagination = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li')
next_button = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li[12]')
total_items = int(driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[2]/li[11]/a').text)

In [6]:
def obtener_items():
    try:
        items = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[1]/li')
        print(f"--total items x pagina {len(items)}")
        return items
    except Exception as e:
        print(f'error al obtener los items')
        print(e)
        return bool([false]);

In [7]:
def traer_datos_item(item, url):
    try:
        
        try:
            s_url = requests.get(url);
            page = BeautifulSoup(s_url.text, 'lxml');
        except Exception as e:
            print('error al acceder al item', s_url)
            print(e)
            return None
        
        container = page.find('div', attrs={'class':'inner-container'}).contents
        
        datos_casa['Sector'] = container[3].contents[0].contents[1].p.get_text().split(',')[0].strip()
        datos_casa['Area'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[2]
        datos_casa['AreaUnidad'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[3]
        datos_casa['Precio'] = container[3].contents[0].contents[2].h3.get_text().replace('.', '').replace('$', '')
        datos_casa['Habitaciones'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[0].get_text()[0:1]
        datos_casa['Baños'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].get_text()[0:1]
        datos_casa['Estrato'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].h2.get_text()[0:1]

        datosPrincipales = container[3].contents[0].contents[7].div.contents;
        
        datosPrincipales

        for dato in datosPrincipales:
            datos_casa[dato.contents[0].get_text()] = dato.contents[1].get_text()

    except Exception as e:
        print(f'error al obtener los items')
        print(e)
 

In [8]:
def pasar_pagina():
    try:
        next_button.click()
        time.sleep(3)
    except Exception as e:
        print(f'error al pasar pagina')
        print(e)

In [10]:
#iniciamos el ciclo principal para la extracción de datos
try:
    print(f'**INICIANDO WEB SCRAPING A metrocuadrado.com**')
    datos = []
    ids = 1
    for numPage in range(total_items):
        print(f'-Obteniendo pagina {numPage + 1}')
        items = obtener_items();
        for item in items:
            item_url = item.find_element(By.XPATH,'.//div/ul/li[1]/a').get_attribute("href");
            datos_casa = {};
            datos_casa['url'] = item_url;
            datos_casa['id'] = ids;
            traer_datos_item(item, item_url)
            datos.append(datos_casa)        
            print(f'-dato {ids}')
            ids = ids + 1;
        pasar_pagina();   
    print(f'**FINAL DEL SCRAPING A metrocuadrado.com**')   
    pd.DataFrame(datos).to_csv('datos.csv')    
except Exception as e:
    print(f'Error recorrer la pagina')
    print(e)
    

        

**INICIANDO WEB SCRAPING A metrocuadrado.com**
-Obteniendo pagina 1
--total items x pagina 50
-dato 1
-dato 2
-dato 3
-dato 4
-dato 5
-dato 6
-dato 7
-dato 8
-dato 9
-dato 10
-dato 11
-dato 12
-dato 13
-dato 14
-dato 15
-dato 16
-dato 17
-dato 18
-dato 19
-dato 20
-dato 21
-dato 22
-dato 23
-dato 24
-dato 25
-dato 26
-dato 27
-dato 28
-dato 29
-dato 30
-dato 31
-dato 32
-dato 33
-dato 34
-dato 35
-dato 36
-dato 37
-dato 38
-dato 39
-dato 40
-dato 41
-dato 42
-dato 43
-dato 44
-dato 45
-dato 46
-dato 47
-dato 48
-dato 49
-dato 50
error al pasar pagina
Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D7A813+48355]
	(No symbol) [0x00D0C4B1]
	(No symbol) [0x00C15358]
	(No symbol) [0x00C1E5A3]
	(No symbol) [0x00C18A78]
	(No symbol) [0x00C17C03]
	(No symbol) [0x00C199E1]
	(No symbol) [0x00C19A80]
	(No symbol) [0x00C41D29]
	(No symbol) [0x00C39DAD]
	(No symbol) [0x00C5A73C]
	(No symbol) [0x00C39A36

-dato 301
-dato 302
-dato 303
-dato 304
-dato 305
-dato 306
-dato 307
-dato 308
-dato 309
-dato 310
-dato 311
-dato 312
-dato 313
-dato 314
-dato 315
-dato 316
-dato 317
-dato 318
-dato 319
-dato 320
-dato 321
-dato 322
-dato 323
-dato 324
-dato 325
-dato 326
-dato 327
-dato 328
-dato 329
-dato 330
-dato 331
-dato 332
-dato 333
-dato 334
-dato 335
-dato 336
-dato 337
-dato 338
-dato 339
-dato 340
-dato 341
-dato 342
-dato 343
-dato 344
-dato 345
-dato 346
-dato 347
-dato 348
-dato 349
-dato 350
error al pasar pagina
Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D7A813+48355]
	(No symbol) [0x00D0C4B1]
	(No symbol) [0x00C15358]
	(No symbol) [0x00C1E5A3]
	(No symbol) [0x00C18A78]
	(No symbol) [0x00C17C03]
	(No symbol) [0x00C199E1]
	(No symbol) [0x00C19A80]
	(No symbol) [0x00C41D29]
	(No symbol) [0x00C39DAD]
	(No symbol) [0x00C5A73C]
	(No symbol) [0x00C39A36]
	(No symbol) [0x00C5AA94]
	(No sy

In [131]:
driver.close()

In [153]:
items = driver.find_elements(By.XPATH,'//*[@id="__next"]/div/div/div[3]/div[2]/div[2]/div[2]/ul[1]/li')
item_url = items[21].find_element(By.XPATH,'.//div/ul/li[1]/a').get_attribute("href")
s_url = requests.get(item_url)
datos_casa = {}
page = BeautifulSoup(s_url.text, 'lxml')
container = page.find('div', attrs={'class':'inner-container'}).contents

datos_casa['Sector'] = container[3].contents[0].contents[1].p.get_text().split(',')[0].strip()
datos_casa['Area'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[2]
datos_casa['AreaUnidad'] = container[3].contents[0].contents[1].p.span.get_text().split(' ')[3]
datos_casa['Precio'] = container[3].contents[0].contents[2].h3.get_text().replace('.', '').replace('$', '')
datos_casa['Habitaciones'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[0].get_text()[0:1]
#datos_casa['Baños'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].get_text()[0:1]
#datos_casa['Estrato'] = container[3].contents[0].contents[3].contents[0].div.contents[1].ul.contents[1].h2.get_text()[0:1]

datosPrincipales = container[3].contents[0].contents[7].div.contents;

for dato in datosPrincipales:
    datos_casa[dato.contents[0].get_text()] = dato.contents[1].get_text()
item_url

'https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-santa-barbara-alta/11074-2810392'